In [1]:
# Setup
import os
from dotenv import load_dotenv
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client.lib.agents.event_logger import EventLogger

# Load configuration
load_dotenv('config.env')
base_url = os.environ.get('LLAMA_STACK_URL', 'http://localhost:8321')
model_id = os.environ.get('LLM_MODEL_ID', 'r1-qwen-14b-w4a16')
temperature = float(os.environ.get("TEMPERATURE", 0.0))
max_tokens = int(os.environ.get("MAX_TOKENS", 4096))

print(f"🔗 Connecting to: {base_url}")
print(f"🤖 Using model: {model_id}")

# Create client
client = LlamaStackClient(base_url=base_url)
print("✅ Client created")


🔗 Connecting to: http://localhost:8321
🤖 Using model: r1-qwen-14b-w4a16
✅ Client created


In [2]:
# Check available tools
print("🔍 Checking MCP Atlassian tools...")
try:
    tools = client.tools.list(toolgroup_id="mcp::atlassian")
    print(f"✅ Found {len(tools)} Atlassian tools")
    
    # Find Jira tools
    jira_tools = [t for t in tools if 'jira' in str(t).lower()]
    print(f"🎫 Jira tools: {len(jira_tools)}")
    
    # Show create and search tools
    for tool in jira_tools:
        tool_name = getattr(tool, 'name', getattr(tool, 'identifier', str(tool)))
        if 'create' in tool_name or 'search' in tool_name:
            print(f"  - {tool_name}")
            
except Exception as e:
    print(f"❌ Error loading tools: {e}")


🔍 Checking MCP Atlassian tools...
✅ Found 37 Atlassian tools
🎫 Jira tools: 27
  - jira_batch_create_issues
  - jira_create_issue
  - jira_create_issue_link
  - jira_create_sprint
  - jira_search
  - jira_search_fields


In [4]:
# Create ReActAgent
print("🤖 Creating ReActAgent...")

# Sampling parameters
sampling_params = {
    "strategy": {"type": "greedy"},
    "max_tokens": max_tokens,
    "temperature": temperature,
}

# Create agent
agent = ReActAgent(
    client=client,
    model=model_id,
    instructions="""You are a helpful assistant that can create and manage Jira issues. 
    Use the available Jira tools to create, search, and manage issues in the KAN project.
    When creating issues, provide clear summaries and descriptions.
    Always use the correct dictionary format for additional_fields, not JSON strings.""",
    tools=["mcp::atlassian"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params=sampling_params,
)

print("✅ ReActAgent created successfully")


🤖 Creating ReActAgent...
✅ ReActAgent created successfully


In [5]:
# Test 1: Simple Jira Issue Creation
print("🧪 Test 1: Create Simple Jira Issue")
print("="*45)

try:
    # Create a session
    session_id = agent.create_session("jira-test-simple")
    print(f"✅ Session created: {session_id}")
    
    # Simple test prompt
    test_prompt = """Create a simple Jira issue in the KAN project with these details:
    - Summary: "Test Issue - ReActAgent Integration"
    - Issue Type: "Task"
    - Description: "This is a test issue created by ReActAgent to verify MCP tool integration works correctly."
    - Priority: Medium
    - Labels: ["test", "react-agent", "mcp"]
    """
    
    print(f"\n📝 Test prompt:")
    print(test_prompt)
    print("\n" + "="*45)
    
    # Create turn with streaming
    response = agent.create_turn(
        messages=[{"role": "user", "content": test_prompt}],
        session_id=session_id,
        stream=True,
        toolgroups=["mcp::atlassian"]  # Explicit toolgroups list
    )
    
    print("\n🚀 Processing with ReActAgent...")
    
    # Process streaming response
    for log in EventLogger().log(response):
        log.print()
        
    print("\n✅ Test 1 completed!")
    
except Exception as e:
    print(f"❌ Test 1 failed: {e}")


🧪 Test 1: Create Simple Jira Issue
✅ Session created: 86d0f0af-f004-4802-bacf-6ad8d2789ef1

📝 Test prompt:
Create a simple Jira issue in the KAN project with these details:
    - Summary: "Test Issue - ReActAgent Integration"
    - Issue Type: "Task"
    - Description: "This is a test issue created by ReActAgent to verify MCP tool integration works correctly."
    - Priority: Medium
    - Labels: ["test", "react-agent", "mcp"]
    


🚀 Processing with ReActAgent...
inference> {
  "thought": "I need to create a Jira issue in the KAN project with the specified details. First, I'll use the create_issue function from the Jira tools. The summary should be 'Test Issue - ReActAgent Integration'. The issue type is 'Task', priority is 'Medium', and labels are 'test', 'react-agent', and 'mcp'. I'll structure the additional_fields correctly, ensuring it's a dictionary, not a JSON string. I'll also include the description as provided. After creating the issue, I'll retrieve its key to confirm it's

In [ ]:
# Test 2: OOM Error Incident Creation
print("\n🧪 Test 2: Create OOM Error Incident")
print("="*40)

try:
    # Create a new session
    session_id = agent.create_session("jira-test-oom")
    print(f"✅ Session created: {session_id}")
    
    # OOM error incident prompt
    oom_prompt = """Create a Jira incident for a pod failing due to OOM (Out of Memory) error in the KAN project:
    
    - Summary: "Pod failing due to OOM error"
    - Issue Type: "Incident"
    - Description: "Pod experiencing Out of Memory errors causing application failures and service disruption. This requires immediate attention."
    - Priority: High
    - Labels: ["oom-error", "pod-failure", "memory-issue", "high-priority", "production-incident"]
    - Components: ["Infrastructure", "Kubernetes"]
    """
    
    print(f"\n📝 OOM Incident prompt:")
    print(oom_prompt)
    print("\n" + "="*40)
    
    # Create turn with streaming
    response = agent.create_turn(
        messages=[{"role": "user", "content": oom_prompt}],
        session_id=session_id,
        stream=True,
        toolgroups=["mcp::atlassian"]
    )
    
    print("\n🚀 Processing OOM incident creation...")
    
    # Process streaming response
    for log in EventLogger().log(response):
        log.print()
        
    print("\n✅ Test 2 completed!")
    
except Exception as e:
    print(f"❌ Test 2 failed: {e}")


In [ ]:
# Test 3: Search for Created Issues
print("\n🧪 Test 3: Search for Created Issues")
print("="*40)

try:
    # Create a new session
    session_id = agent.create_session("jira-test-search")
    print(f"✅ Session created: {session_id}")
    
    # Search prompt
    search_prompt = """Search for recently created issues in the KAN project to verify the issues were created successfully.
    Look for issues created today with keywords like 'test', 'OOM', 'ReActAgent', or 'pod'."""
    
    print(f"\n📝 Search prompt:")
    print(search_prompt)
    print("\n" + "="*40)
    
    # Create turn with streaming
    response = agent.create_turn(
        messages=[{"role": "user", "content": search_prompt}],
        session_id=session_id,
        stream=True,
        toolgroups=["mcp::atlassian"]
    )
    
    print("\n🚀 Processing search...")
    
    # Process streaming response
    for log in EventLogger().log(response):
        log.print()
        
    print("\n✅ Test 3 completed!")
    
except Exception as e:
    print(f"❌ Test 3 failed: {e}")
